In [1]:
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import f1_score, hamming_loss, make_scorer, accuracy_score

from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.model_selection.measures import get_combination_wise_output_matrix
from skmultilearn.model_selection import iterative_train_test_split


In [2]:
DATA_DIR = "../../data/processed/"
INPUT_FILE_NAME = 'cleaned_squashed1.parquet'
df = pd.read_parquet(DATA_DIR + INPUT_FILE_NAME)
df.head()

,speaker,headline,description,duration,tags,transcript,WC,clean_transcript,clean_transcript_string,squash_tags
0,Al Gore,Averting the climate crisis,With the same humor and humanity he exuded in ...,0:16:17,"cars,alternative energy,culture,politics,scien...","0:14\r\r\rThank you so much, Chris.\rAnd it's ...",2281.0,"[thank, chris, truly, great, honor, opportunit...",thank chris truly great honor opportunity come...,"culture,politics,science,climate change,enviro..."
1,Amy Smith,Simple designs to save a life,Fumes from indoor cooking fires kill more than...,0:15:06,"MacArthur grant,simplicity,industrial design,a...","0:11\r\r\rIn terms of invention,\rI'd like to ...",2687.0,"[term, invention, like, tell, tale, favorite, ...",term invention like tell tale favorite project...,"invention,engineering,design,global issues"
2,Ashraf Ghani,How to rebuild a broken state,Ashraf Ghani's passionate and powerful 10-minu...,0:18:45,"corruption,poverty,economics,investment,milita...","0:12\r\r\rA public, Dewey long ago observed,\r...",2506.0,"[public, dewey, long, ago, observe, constitute...",public dewey long ago observe constitute discu...,"poverty,economics,culture,politics,policy,glob..."
3,Burt Rutan,The real future of space exploration,"In this passionate talk, legendary spacecraft ...",0:19:37,"aircraft,flight,industrial design,NASA,rocket ...","0:11\r\r\rI want to start off by saying, Houst...",3092.0,"[want, start, say, houston, problem, enter, se...",want start say houston problem enter second ge...,"invention,engineering,entrepreneur,design,busi..."
4,Chris Bangle,Great cars are great art,American designer Chris Bangle explains his ph...,0:20:04,"cars,industrial design,transportation,inventio...","0:12\r\r\rWhat I want to talk about is, as bac...",3781.0,"[want, talk, background, idea, car, art, actua...",want talk background idea car art actually mea...,"invention,design,technology,business,art"


In [3]:
df = df.dropna(subset=['squash_tags'])
df = df.reset_index(drop=True)
df.iloc[:,:10].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2382 entries, 0 to 2381
Data columns (total 10 columns):
speaker                    2382 non-null object
headline                   2382 non-null object
description                2382 non-null object
duration                   2382 non-null object
tags                       2382 non-null object
transcript                 2382 non-null object
WC                         2382 non-null float64
clean_transcript           2382 non-null object
clean_transcript_string    2382 non-null object
squash_tags                2382 non-null object
dtypes: float64(1), object(9)
memory usage: 186.2+ KB


In [4]:
X = df['clean_transcript_string']
labels = df[['squash_tags']]

In [5]:
from sklearn.preprocessing import MultiLabelBinarizer

y = []
for index, row in labels.iterrows():
    y.append(set(row['squash_tags'].split(',')))
    
mlb = MultiLabelBinarizer()
encoded_y = mlb.fit_transform(y)

In [6]:
print(encoded_y[0])
print(len(encoded_y[0]))
print(mlb.inverse_transform(encoded_y)[:10])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0]
100
[('climate change', 'culture', 'environment', 'global issues', 'politics', 'science', 'sustainability', 'technology'), ('design', 'engineering', 'global issues', 'invention'), ('business', 'culture', 'economics', 'entrepreneur', 'global development', 'global issues', 'policy', 'politics', 'poverty'), ('business', 'design', 'engineering', 'entrepreneur', 'invention'), ('art', 'business', 'design', 'invention', 'technology'), ('biodiversity', 'biology', 'biotech', 'ecology', 'entrepreneur', 'genetics', 'invention', 'oceans', 'science', 'technology'), ('computers', 'entertainment', 'media', 'music', 'performance', 'technology'), ('architecture', 'cities', 'collaboration', 'culture', 'design'), ('business', 'education', 'innovation', 'invention', 'robots', 'science', 'social change', '

In [7]:
from skmultilearn.model_selection import iterative_train_test_split

X_train, y_train, X_test, y_test = iterative_train_test_split(X.values.reshape(len(X.values), 1), encoded_y, test_size = 0.2)
X_train = pd.DataFrame(X_train)[0]
X_test = pd.DataFrame(X_test)[0]

In [8]:
print(y_train.sum(axis=0))
print(y_test.sum(axis=0))

[112  38  73  83  61 160  48  52  46 137  55 115 256 102  88  65  41 122
 144 109  82 135 375  70  35 316  56  42 122 159  37  84 228  48 121  49
  74  42  52  40 139  47  46 377  61  54  53 173 100  99 132  71  76  40
  64 161  59 101  42  81  76  49  40  49  66 120  42  51  44  95  86  66
  55  40  37  76  66  42  69  53  34 138  56  42  76  40  44 399 149 159
  41  88  54 532  49  61  79  85  48  46]
[ 35  10  21  25  15  44   8  11  14  37  15  33  73  33  25  16  14  41
  41  27  23  39  95  19  17  84  16  10  32  47  14  23  57  12  34  15
  22   7  14  15  42  12  12 106  22  17  18  53  30  36  32  28  30  10
  22  51  18  35  16  26  23  11  13  17  20  34  17  13  17  24  20  18
  18   9  15  22  19  18  19  14  15  45  14   9  28  12  11 123  49  43
  15  24  20 163  16  19  29  30  21  16]


## Gridsearch for the best single model for all labels

### References 
http://scikit.ml/api/skmultilearn.problem_transform.br.html

https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

http://scikit.ml/stratification.html

https://stackoverflow.com/questions/12632992/gridsearch-for-an-estimator-inside-a-onevsrestclassifier/12637528#12637528

### Binary Relevance

In [9]:
# TODO: 
# 1. Check if TfidfTransformer use_idf=False is the same as Countvectorizer? or there are other metrics to suppress
# 2. Get scoring function to work, hamming? -- kinda done
# 3. Balanced class labels
# 4. Set better param ranges
# 5. Remove vectorizer step once we decide on which is better, then use sparse csr and hopefully it trains faster

# param_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
param_range = [1, 2, 3, 4, 5]
param_range_lr = [1.0, 0.5, 0.1]

# Set params, comment out as see fit

vectorizer_params = {
#     'vectorizer__min_df': np.linspace(0.005, 0.05, 5),
#     'vectorizer__ngram_range': [(1, 1), (1, 2)], # This shit blows up your memory
#     'tfidf__norm': ('l1', 'l2'),
    'tfidf__use_idf': [True, False],
}

lr_params = {
    'clf__classifier': [LogisticRegression()],
        'clf__classifier__penalty': ['l1', 'l2'],
        'clf__classifier__C': param_range_lr,
        'clf__classifier__solver': ['liblinear'],
        'clf__classifier__class_weight': ['balanced'],
}

svc_params = {
    'clf__classifier': [SVC()],
        'clf__classifier__kernel': ['linear', 'rbf'],
        'clf__classifier__C': param_range, # np.logspace(-1, 2, 10),
        'clf__classifier__gamma': ['auto'], # np.logspace(-1, 1, 10)
        'clf__classifier__probability': [True],
        'clf__classifier__class_weight': ['balanced'],
}

rf_params = {
    'clf__classifier': [RandomForestClassifier()],
        'clf__classifier__criterion': ['gini', 'entropy'],
        'clf__classifier__min_samples_leaf': param_range,
        'clf__classifier__max_depth': param_range,
        'clf__classifier__min_samples_split': param_range[1:],
        'clf__classifier__n_estimators': [10],
        'clf__classifier__class_weight': ['balanced'],
}

mnb_params = {
    'clf__classifier': [MultinomialNB()],
        'clf__classifier__alpha': [0.7, 1.0],
}

## Stack params
parameters = [
    {**vectorizer_params, **lr_params},
#     {**vectorizer_params, **svc_params},
#     {**vectorizer_params, **rf_params},
#     {**vectorizer_params, **mnb_params}
]

br_pipeline = Pipeline([('vectorizer', CountVectorizer()),
                        ('tfidf', TfidfTransformer()),
                        ('clf', BinaryRelevance()),
                       ]
                      )

# Gridsearch settings
# scoring = make_scorer(f1_score, average='micro') # possible scorings 'f1_micro' 'f1_macro'
# scoring = 'f1_micro'
# scoring = make_scorer(hamming_loss)
# scoring = 'neg_log_loss'
scoring = 'f1_samples'
folds = 3
njobs = -1

br_model = GridSearchCV(br_pipeline, parameters, scoring=scoring, cv=folds, n_jobs=njobs)

In [10]:
%%time
br_model.fit(X_train,y_train)
print(br_model.best_params_, br_model.best_score_)
pd.DataFrame(br_model.cv_results_)

{'clf__classifier': LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False), 'clf__classifier__C': 1.0, 'clf__classifier__class_weight': 'balanced', 'clf__classifier__penalty': 'l2', 'clf__classifier__solver': 'liblinear', 'tfidf__use_idf': False} 0.38711631250803646
Wall time: 12min 14s


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__classifier,param_clf__classifier__C,param_clf__classifier__class_weight,param_clf__classifier__penalty,param_clf__classifier__solver,param_tfidf__use_idf,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,111.470159,6.446176,35.510447,1.481652,"LogisticRegression(C=1.0, class_weight='balanc...",1,balanced,l1,liblinear,True,"{'clf__classifier': LogisticRegression(C=1.0, ...",0.345882,0.357421,0.377387,0.360222,0.013015,6
1,111.614353,7.337566,36.533854,1.216782,"LogisticRegression(C=1.0, class_weight='balanc...",1,balanced,l1,liblinear,False,"{'clf__classifier': LogisticRegression(C=1.0, ...",0.335754,0.330314,0.371690,0.345914,0.018354,8
2,91.772715,7.854513,34.705964,3.954210,"LogisticRegression(C=1.0, class_weight='balanc...",1,balanced,l2,liblinear,True,"{'clf__classifier': LogisticRegression(C=1.0, ...",0.379193,0.407101,0.361440,0.382576,0.018789,4
3,101.763127,5.900635,35.298412,2.738021,"LogisticRegression(C=1.0, class_weight='balanc...",1,balanced,l2,liblinear,False,"{'clf__classifier': LogisticRegression(C=1.0, ...",0.383596,0.383091,0.394667,0.387116,0.005341,1
4,111.945456,3.176044,34.163979,1.421262,"LogisticRegression(C=1.0, class_weight='balanc...",0.5,balanced,l1,liblinear,True,"{'clf__classifier': LogisticRegression(C=1.0, ...",0.324174,0.329758,0.359938,0.337949,0.015708,9
5,112.323486,9.113564,34.014516,0.654730,"LogisticRegression(C=1.0, class_weight='balanc...",0.5,balanced,l1,liblinear,False,"{'clf__classifier': LogisticRegression(C=1.0, ...",0.312190,0.299764,0.360455,0.324130,0.026171,10
6,96.165212,5.961420,34.389342,2.365571,"LogisticRegression(C=1.0, class_weight='balanc...",0.5,balanced,l2,liblinear,True,"{'clf__classifier': LogisticRegression(C=1.0, ...",0.381965,0.402569,0.366082,0.383538,0.014933,2
7,99.175079,4.030367,35.096197,0.715753,"LogisticRegression(C=1.0, class_weight='balanc...",0.5,balanced,l2,liblinear,False,"{'clf__classifier': LogisticRegression(C=1.0, ...",0.374150,0.365378,0.394179,0.377900,0.012051,5
8,97.681648,0.635577,32.685025,0.959675,"LogisticRegression(C=1.0, class_weight='balanc...",0.1,balanced,l1,liblinear,True,"{'clf__classifier': LogisticRegression(C=1.0, ...",0.239478,0.238232,0.268744,0.248813,0.014097,11
9,99.262824,6.947729,33.324450,1.467823,"LogisticRegression(C=1.0, class_weight='balanc...",0.1,balanced,l1,liblinear,False,"{'clf__classifier': LogisticRegression(C=1.0, ...",0.219493,0.212393,0.275166,0.235675,0.028063,12


In [11]:
y_pred_prob = br_model.predict_proba(X_test)
t = 0.1 # threshold value
y_pred_new = (y_pred_prob >= t).astype(int)
score = f1_score(y_test, y_pred_new, average="micro")
print(f"Binary relevance best model's f1-score {score}")

Binary relevance best model's f1-score 0.1306289367302811


### OneVsRest

In [12]:
# TODO: 
# 1. Check if TfidfTransformer use_idf=False is the same as Countvectorizer? or there are other metrics to suppress
# 2. Use proper scoring function - ideally, predicting relevant labels should be more important than predicting irrelevant ones
# 3. Balanced class labels
# 4. Set better param ranges

# param_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
param_range = [1, 2, 3, 4, 5]
param_range_lr = [1.0, 0.5, 0.1]

# Set params, comment out as see fit

vectorizer_params = {
#     'vectorizer__min_df': np.linspace(0.005, 0.05, 5),
#     'vectorizer__ngram_range': [(1, 1), (1, 2)], # This shit blows up your memory
#     'tfidf__norm': ('l1', 'l2'),
    'tfidf__use_idf': [True, False],
}

lr_params = {
    'clf__estimator': [LogisticRegression()],
        'clf__estimator__penalty': ['l1', 'l2'],
        'clf__estimator__C': param_range_lr,
        'clf__estimator__solver': ['liblinear'],
        'clf__estimator__class_weight': ['balanced'],
}

svc_params = {
    'clf__estimator': [SVC()],
        'clf__estimator__kernel': ['linear', 'rbf'],
        'clf__estimator__C': param_range, # np.logspace(-1, 2, 10),
        'clf__estimator__gamma': ['auto'], # np.logspace(-1, 1, 10)
        'clf__estimator__probability': [True],
        'clf__estimator__class_weight': ['balanced'],
}

rf_params = {
    'clf__estimator': [RandomForestClassifier()],
        'clf__estimator__criterion': ['gini', 'entropy'],
        'clf__estimator__min_samples_leaf': param_range,
        'clf__estimator__max_depth': param_range,
        'clf__estimator__min_samples_split': param_range[1:],
        'clf__estimator__n_estimators': [10],
        'clf__estimator__class_weight': ['balanced'],
}

mnb_params = {
    'clf__estimator': [MultinomialNB()],
        'clf__estimator__alpha': [0.7, 1.0],
}

## Stack params
parameters = [
#     {**vectorizer_params, **lr_params},
#     {**vectorizer_params, **svc_params},
#     {**vectorizer_params, **rf_params},
    {**vectorizer_params, **mnb_params}
]

ovr_pipeline = Pipeline([('vectorizer', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', OneVsRestClassifier(LogisticRegression())),
                        ]
                       )

# Gridsearch settings
# scoring = make_scorer(f1_score, average='micro') # possible scorings 'f1_micro' 'f1_macro'
scoring = 'f1_micro'
# scoring = make_scorer(hamming_loss) # hamming gives equal weighting to both relevant and irrelevant?
# maybe use precision somewhere
folds = 3
njobs = -1

ovr_model = GridSearchCV(ovr_pipeline, parameters, scoring=scoring, cv=folds, n_jobs=njobs)

In [13]:
%%time
ovr_model.fit(X_train,y_train)
print(ovr_model.best_params_, ovr_model.best_score_)
pd.DataFrame(ovr_model.cv_results_)

{'clf__estimator': MultinomialNB(alpha=0.7, class_prior=None, fit_prior=True), 'clf__estimator__alpha': 0.7, 'tfidf__use_idf': True} 0.0
Wall time: 17.3 s


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__estimator,param_clf__estimator__alpha,param_tfidf__use_idf,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,4.235401,0.245698,1.569807,0.213995,"MultinomialNB(alpha=0.7, class_prior=None, fit...",0.7,True,"{'clf__estimator': MultinomialNB(alpha=0.7, cl...",0.0,0.0,0.0,0.0,0.0,1
1,4.178484,0.325424,1.532880,0.138228,"MultinomialNB(alpha=0.7, class_prior=None, fit...",0.7,False,"{'clf__estimator': MultinomialNB(alpha=0.7, cl...",0.0,0.0,0.0,0.0,0.0,1
2,3.883495,0.666608,1.407283,0.233621,"MultinomialNB(alpha=0.7, class_prior=None, fit...",1,True,"{'clf__estimator': MultinomialNB(alpha=0.7, cl...",0.0,0.0,0.0,0.0,0.0,1
3,2.945858,0.022153,1.061989,0.068123,"MultinomialNB(alpha=0.7, class_prior=None, fit...",1,False,"{'clf__estimator': MultinomialNB(alpha=0.7, cl...",0.0,0.0,0.0,0.0,0.0,1


In [14]:
y_pred_prob = ovr_model.predict_proba(X_test)
t = 0.1 # threshold value
y_pred_new = (y_pred_prob >= t).astype(int)
score = f1_score(y_test, y_pred_new, average="micro")
print(f"One vs Rest best model's f1-score {score}")

One vs Rest best model's f1-score 0.020272631946871723


## Gridsearch best model for each tag

https://stackoverflow.com/questions/38555650/try-multiple-estimator-in-one-grid-search



In [15]:
# param_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
param_range = [1, 2, 3, 4, 5]
param_range_lr = [1.0, 0.5, 0.1]

# Set params, comment out as see fit

vectorizer_params = {
#     'vectorizer__min_df': np.linspace(0.005, 0.05, 5),
#     'vectorizer__ngram_range': [(1, 1), (1, 2)], # This shit blows up your memory
#     'tfidf__norm': ('l1', 'l2'),
    'tfidf__use_idf': [True, False],
}

# Add any Binary classification model setting here.
# Also add to general parameters to be passed into pipeline below if want to use new model.

lr_params = {
    'clf': [LogisticRegression()],
        'clf__penalty': ['l1', 'l2'],
        'clf__C': param_range_lr,
        'clf__solver': ['liblinear'],
        'clf__class_weight': ['balanced'],
}

svc_params = {
    'clf': [SVC()],
        'clf__kernel': ['linear', 'rbf'],
        'clf__C': param_range, # np.logspace(-1, 2, 10),
        'clf__gamma': ['auto'], # np.logspace(-1, 1, 10)
        'clf__probability': [True],
        'clf__class_weight': ['balanced'],
}

rf_params = {
    'clf': [RandomForestClassifier()],
        'clf__criterion': ['gini', 'entropy'],
#         'clf__min_samples_leaf': param_range,
#         'clf__max_depth': param_range,
        'clf__min_samples_split': param_range[1:],
        'clf__n_estimators': [15],
        'clf__class_weight': ['balanced'],
}

mnb_params = {
    'clf': [MultinomialNB()],
        'clf__alpha': [0.7, 1.0],
}

## Stack params
parameters = [
#     {**vectorizer_params, **lr_params},
#     {**vectorizer_params, **svc_params},
    {**vectorizer_params, **rf_params},
#     {**vectorizer_params, **mnb_params}
]

per_tag_pipe = Pipeline([('vectorizer', CountVectorizer()), 
                  ('tfidf', TfidfTransformer()), 
                  ('clf', LogisticRegression())])

# scoring = make_scorer(hamming_loss)
scoring = 'f1'
# scoring = 'f1_micro'
# scoring = 'balanced_accuracy'
# scoring = 'precision'
folds = 3
njobs = -1

per_tag_model = GridSearchCV(per_tag_pipe, parameters, scoring=scoring, cv=folds, n_jobs=njobs)

In [16]:
tags = [tag for tag in mlb.inverse_transform(np.ones(shape=(1, 100)))[0]]
print(tags)
tags.index('technology')

['activism', 'adventure', 'africa', 'animals', 'architecture', 'art', 'beauty', 'big problems', 'biodiversity', 'biology', 'biotech', 'brain', 'business', 'children', 'cities', 'climate change', 'cognitive science', 'collaboration', 'communication', 'community', 'computers', 'creativity', 'culture', 'data', 'demo', 'design', 'disease', 'ecology', 'economics', 'education', 'energy', 'engineering', 'entertainment', 'entrepreneur', 'environment', 'evolution', 'exploration', 'family', 'film', 'food', 'future', 'genetics', 'global development', 'global issues', 'government', 'green', 'happiness', 'health', 'health care', 'history', 'humanity', 'humor', 'identity', 'illness', 'inequality', 'innovation', 'internet', 'invention', 'language', 'leadership', 'life', 'live music', 'math', 'media', 'medical research', 'medicine', 'mental health', 'mind', 'motivation', 'music', 'nature', 'neuroscience', 'oceans', 'parenting', 'peace', 'performance', 'personal growth', 'philosophy', 'photography', 'p

93

In [17]:
for index in range(91, 95): #range(len(tags))
    print(f"Processing {tags[index]}")
    per_tag_model.fit(X_train, y_train[:, index])
#     display(pd.DataFrame(per_tag_model.cv_results_))
    t = 0.2 #threshold value
    prediction_prob = per_tag_model.predict_proba(X_test)
    prediction = (prediction_prob[:, 1] >= t).astype(int)
    # save model or model params somewhere
    print(f'tag {index}: {tags[index]} best model {per_tag_model.best_params_}')
    print(f'tag {index}: {tags[index]} counts - predicted: {sum(prediction)}, actual: {sum(y_test[:, index])}')
    print(f'tag {index}: {tags[index]} test f1-score is {f1_score(y_test[:, index], prediction, average="binary")}')
    print(f'tag {index}: {tags[index]} test accuracy is {accuracy_score(y_test[:, index], prediction)}')
    print('--------------------------')

Processing storytelling
tag 91: storytelling best model {'clf': RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=3, min_weight_fraction_leaf=0.0,
                       n_estimators=15, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False), 'clf__class_weight': 'balanced', 'clf__criterion': 'gini', 'clf__min_samples_split': 3, 'clf__n_estimators': 15, 'tfidf__use_idf': True}
tag 91: storytelling counts - predicted: 31, actual: 24
tag 91: storytelling test f1-score is 0.14545454545454548
tag 91: storytelling test accuracy is 0.9182608695652174
--------------------------
Processing sustainability
tag 92: sustainability best model {'clf': RandomForestClassifier(bootstrap=Tru